INTEGRANTES DO GRUPO 41

Vinicius dos Santos Mendes

trekpsn@gmail.com

Pedro Henrique de Oliveira Carrijo

pedro.car.rijo@hotmail.com

DIOGO MORALES MARTINS HALAS

diogo.halas@gmail.com

Diogo Paiva

paiva.721@gmail.com

Jefferson Vega Thurck Junior

jeffersonthurck@gmail.com

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
pip install opencv-python-headless deepface tf-keras tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=25e8e95c6a69564bbb3b3e945113681b34d2fa5b00a0ab73e993c6609836200c
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [3]:
pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=3427e2412257020bb19e1194b5413890091e218861a955cfa78e1cd27f34b5a5
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [4]:
pip install opencv-python mediapipe tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 18.1 MB/s eta 0:00:00


In [5]:
import cv2
import face_recognition
import os
import numpy as np
from tqdm import tqdm
from deepface import DeepFace
import mediapipe as mp
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from collections import deque

# Função para carregar imagens e codificações faciais de uma pasta
def load_images_from_folder(folder):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                face_encoding = face_encodings[0]
                name = os.path.splitext(filename)[0][:-1]  # Remove sufixo numérico e extensão
                known_face_encodings.append(face_encoding)
                known_face_names.append(name)

    return known_face_encodings, known_face_names

def calculate_movement_speed(prev_landmarks, curr_landmarks, mp_pose):
    """Calcula a velocidade de movimento entre frames."""
    if prev_landmarks is None or curr_landmarks is None:
        return 0

    # Pontos chave para monitorar (ombros e cotovelos)
    key_points = [
        mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        mp_pose.PoseLandmark.LEFT_ELBOW.value,
        mp_pose.PoseLandmark.RIGHT_ELBOW.value
    ]

    total_speed = 0
    for point in key_points:
        prev_point = prev_landmarks[point]
        curr_point = curr_landmarks[point]

        # Calcula a distância euclidiana entre as posições
        speed = np.sqrt(
            (curr_point.x - prev_point.x)**2 +
            (curr_point.y - prev_point.y)**2
        )
        total_speed += speed

    return total_speed / len(key_points)

def is_movement_anomalous(speed, speed_history, threshold_multiplier=2.0):
    """Determina se um movimento é anômalo baseado no histórico de velocidades."""
    if len(speed_history) < 10:  # Precisamos de um histórico mínimo
        return False

    mean_speed = np.mean(speed_history)
    std_speed = np.std(speed_history)

    # Um movimento é considerado anômalo se estiver muito acima da média histórica
    threshold = mean_speed + (threshold_multiplier * std_speed)
    return speed > threshold

def detect_arm_movements(landmarks, mp_pose):
    """Detecta movimentos dos braços usando landmarks do MediaPipe."""
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    # Verifica se os braços estão levantados (acima dos ombros)
    left_arm_up = left_elbow.y < left_shoulder.y or left_wrist.y < left_shoulder.y
    right_arm_up = right_elbow.y < right_shoulder.y or right_wrist.y < right_shoulder.y

    return {
        "left_arm_up": left_arm_up,
        "right_arm_up": right_arm_up
    }

def detect_faces_and_emotions(video_path, output_path, known_face_encodings, known_face_names):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps, total_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Inicializa o MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    # Configura o gravador de vídeo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    movements_log = []
    emotions_log = []
    analyzed_frames = 0
    anomalies = 0

    # Variáveis para detecção de movimentos anômalos
    prev_landmarks = None
    speed_history = deque(maxlen=30)  # Mantém histórico dos últimos 30 frames
    movement_pattern = deque(maxlen=5)  # Mantém padrão de movimentos recentes

    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, frame = cap.read()
        if not ret:
            break

        analyzed_frames += 1
        frame_anomalies = []  # Anomalias detectadas no frame atual

        # Detecção de pose e análise de movimento
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(rgb_frame)

        if pose_results.pose_landmarks:
            # Calcula velocidade do movimento
            current_speed = calculate_movement_speed(
                prev_landmarks,
                pose_results.pose_landmarks.landmark,
                mp_pose
            )
            speed_history.append(current_speed)

            # Verifica se o movimento é anômalo
            if is_movement_anomalous(current_speed, list(speed_history)):
                anomalies += 1
                frame_anomalies.append("Movimento brusco detectado")
                # Desenha alerta no frame
                cv2.putText(frame, "MOVIMENTO ANOMALO!", (10, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            prev_landmarks = pose_results.pose_landmarks.landmark

            # Detecta movimentos dos braços
            movements = detect_arm_movements(pose_results.pose_landmarks.landmark, mp_pose)

            # Registra o movimento atual
            if movements["left_arm_up"] and movements["right_arm_up"]:
                current_movement = "Ambos os braços levantados"
            elif movements["left_arm_up"]:
                current_movement = "Braco esquerdo levantado"
            elif movements["right_arm_up"]:
                current_movement = "Braco direito levantado"
            else:
                current_movement = "Bracos abaixados"

            movement_pattern.append(current_movement)
            movements_log.append(current_movement)

            # Verifica padrões anômalos de movimento
            if len(movement_pattern) == 5:
                if len(set(movement_pattern)) >= 4:  # Muitas mudanças rápidas
                    frame_anomalies.append("Padrão de movimento errático")
                    anomalies += 1

            # Desenha as landmarks do pose
            mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Exibe informações no frame
            cv2.putText(frame, f'Movimento: {current_movement}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Detecção de emoções com DeepFace
        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            # Reconhecimento de face
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame,model="hog")
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            face_names = []

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Desconhecido"
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)

            if result:
                for face in result:
                    emotions_log.append(face['dominant_emotion'])

                    # Desenha informações no frame
                    x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, face['dominant_emotion'], (x, y-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

                    # Associar a face detectada pelo DeepFace com as faces conhecidas
                    for (top, right, bottom, left), name in zip(face_locations, face_names):
                        if x <= left <= x + w and y <= top <= y + h:
                            # Escrever o nome abaixo da face
                            cv2.putText(frame, name, (x + 6, y + h - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                            break
        except Exception as e:
            print(f"Erro na análise DeepFace: {e}")

        # Adiciona informações ao frame
        if frame_anomalies:
            anomaly_text = " | ".join(frame_anomalies)
            cv2.putText(frame, f'Anomalias: {anomaly_text}', (10, 90),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    pose.close()

    return movements_log, emotions_log, analyzed_frames, anomalies

def categorizador(movements_log):
    """Categoriza os movimentos detectados."""
    texts = [
        "Ambos os braços levantados", "Braço esquerdo levantado", "Braço direito levantado", "Braços abaixados",
        "Ambos os braços levantados", "Braço direito levantado", "Braço esquerdo levantado", "Braços abaixados",
        "Braço esquerdo levantado", "Ambos os braços levantados", "Braço direito levantado", "Braços abaixados",
        "Braço direito levantado", "Ambos os braços levantados", "Braço esquerdo levantado", "Braços abaixados"
    ]

    labels = [
        "levantar_ambos", "levantar_esquerdo", "levantar_direito", "repouso",
        "levantar_ambos", "levantar_direito", "levantar_esquerdo", "repouso",
        "levantar_esquerdo", "levantar_ambos", "levantar_direito", "repouso",
        "levantar_direito", "levantar_ambos", "levantar_esquerdo", "repouso"
    ]

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

    # Criar um pipeline de transformação de texto e classificação
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())

    # Treinar o modelo
    model.fit(X_train, y_train)

    # Fazer previsões no conjunto de teste
    predicted_labels = model.predict(X_test)

    # Avaliar o modelo
    print(metrics.classification_report(y_test, predicted_labels, zero_division=0))

    # Classificar novos textos
    predicted_new_labels = model.predict(movements_log)
    text_categorizador = [f"{movement}-{label}" for movement, label in zip(movements_log, predicted_new_labels)]

    # Transformando em uma única string
    text_completo = " ".join(text_categorizador)
    print(text_completo)
    return text_completo, predicted_new_labels

def resumidor(movements_log, emotions_log, labels, analyzed_frames, anomalies):
    """Gera um resumo completo da análise."""
    # Resumo das Ações Detectadas
    action_frequencies = {}
    for action in set(movements_log):
        action_frequencies[action] = movements_log.count(action) / len(movements_log) * 100
    action_summary = "Resumo das Ações Detectadas:\n"
    for action, frequency in action_frequencies.items():
        action_summary += f"- {action}: {frequency:.2f}% do tempo\n"

    # Resumo das Emoções Detectadas
    emotion_frequencies = {}
    for emotion in set(emotions_log):
        emotion_frequencies[emotion] = emotions_log.count(emotion) / len(emotions_log) * 100
    emotion_summary = "Resumo das Emoções Detectadas:\n"
    for emotion, frequency in emotion_frequencies.items():
        emotion_summary += f"- {emotion}: {frequency:.2f}% do tempo\n"

    # Resumo das Labels Detectadas
    labels_frequencies = {}
    for label in set(labels):
        labels_frequencies[label] = labels.count(label) / len(labels) * 100
    label_summary = "Resumo das Labels Detectadas:\n"
    for label, frequency in labels_frequencies.items():
        label_summary += f"- {label}: {frequency:.2f}% do tempo\n"

    # Adicionar informações sobre frames e anomalias
    frame_summary = f"\nInformações de Análise:\n"
    frame_summary += f"- Total de frames analisados: {analyzed_frames}\n"
    frame_summary += f"- Número de movimentos anômalos detectados: {anomalies}\n"
    frame_summary += f"- Porcentagem de frames com anomalias: {(anomalies/analyzed_frames*100):.2f}%\n"

    return action_summary, emotion_summary, label_summary, frame_summary

# Código principal
def main():
    script_dir = "/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE 4/"
    #script_dir = "/content/drive/MyDrive/FASE 4/"
    image_folder = os.path.join(script_dir, 'images')
    known_face_encodings, known_face_names = load_images_from_folder(image_folder)

    input_video_path = os.path.join(script_dir, 'video-tech.mp4')
    output_video_path = os.path.join(script_dir, 'output_video_recognize.mp4')

    # Executa a análise do vídeo
    movements_log, emotions_log, analyzed_frames, anomalies = detect_faces_and_emotions(
        input_video_path, output_video_path, known_face_encodings, known_face_names)

    # Categoriza os movimentos
    text_completo, labels = categorizador(movements_log)

    # Gera o resumo
    action_summary, emotion_summary, label_summary, frame_summary = resumidor(
        movements_log, emotions_log, list(labels), analyzed_frames, anomalies)

    # Imprime os result
    print(action_summary)
    print(emotion_summary)
    print(label_summary)
    print(frame_summary)

main()

24-11-22 14:58:07 - Directory /root/.deepface has been created
24-11-22 14:58:07 - Directory /root/.deepface/weights has been created


Processando vídeo:   0%|          | 0/3326 [00:00<?, ?it/s]

24-11-22 14:59:53 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

100%|██████████| 5.98M/5.98M [00:00<00:00, 89.8MB/s]
Processando vídeo: 100%|██████████| 3326/3326 [1:12:28<00:00,  1.31s/it]

                   precision    recall  f1-score   support

   levantar_ambos       1.00      1.00      1.00         1
 levantar_direito       1.00      1.00      1.00         1
levantar_esquerdo       1.00      1.00      1.00         2

         accuracy                           1.00         4
        macro avg       1.00      1.00      1.00         4
     weighted avg       1.00      1.00      1.00         4

Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos abaixados-repouso Bracos ab

In [ ]:
import cv2
import face_recognition
import os
import numpy as np
from tqdm import tqdm
from deepface import DeepFace
import mediapipe as mp
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Função para carregar imagens e codificações faciais de uma pasta
def load_images_from_folder(folder):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                face_encoding = face_encodings[0]
                name = os.path.splitext(filename)[0][:-1]  # Remove sufixo numérico e extensão
                known_face_encodings.append(face_encoding)
                known_face_names.append(name)

    return known_face_encodings, known_face_names

# Função para detectar movimentos dos braços
def detect_arm_movements(landmarks, mp_pose):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

    left_arm_up = left_elbow.y < left_shoulder.y
    right_arm_up = right_elbow.y < right_shoulder.y

    return {
        "left_arm_up": left_arm_up,
        "right_arm_up": right_arm_up
    }


# Função principal para processar vídeo e detectar emoções e movimentos
def detect_faces_and_emotions(video_path, output_path, known_face_encodings, known_face_names):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps, total_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Inicializa o MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    # Configura o gravador de vídeo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    movements_log, labels_log, emotions_log = [], [], []

    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, frame = cap.read()
        if not ret:
            break

        # Detecção de emoções com DeepFace
        try:
          result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        except Exception as e:
            print(f"Erro na análise DeepFace: {e}")
            result = []
        # Reconhecimento de face
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame,model="hog")
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        face_names = []

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Desconhecido"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            face_names.append(name)

        # Iterar sobre cada face detectada pelo DeepFace
        for face in result:
            # Obter a caixa delimitadora da face
            x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']

            # Obter a emoção dominante
            dominant_emotion = face['dominant_emotion']
            emotions_log.append(dominant_emotion)
            # Desenhar um retângulo ao redor da face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Escrever a emoção dominante acima da face
            cv2.putText(frame, dominant_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

            # Associar a face detectada pelo DeepFace com as faces conhecidas
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                if x <= left <= x + w and y <= top <= y + h:
                    # Escrever o nome abaixo da face
                    cv2.putText(frame, name, (x + 6, y + h - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                    break

            # Converter o frame para RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Detecção de poses e movimentos de braços
            results = pose.process(rgb_frame)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                movements = detect_arm_movements(results.pose_landmarks.landmark, mp_pose)

                # Registrar movimentos e labels
                if movements["left_arm_up"] and movements["right_arm_up"]:
                    movements_log.append("Ambos os braços levantados")
                    #labels_log.append("levantar_ambos")
                elif movements["left_arm_up"]:
                    movements_log.append("Braco esquerdo levantado")
                    #labels_log.append("levantar_esquerdo")
                elif movements["right_arm_up"]:
                    movements_log.append("Braco direito levantado")
                    #labels_log.append("levantar_direito")
                else:
                    movements_log.append("Bracos abaixados")
                    #labels_log.append("repouso")
                # Exibir a contagem de movimentos dos braços no frame
                cv2.putText(frame, f'Movimentos dos bracos: {movements_log[-1]}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)

        out.write(frame)

    cap.release()
    out.release()

    return movements_log, emotions_log

# Função para categorizar movimentos
def categorizador(movements_log):
    texts = [
        "Ambos os braços levantados", "Braço esquerdo levantado", "Braço direito levantado", "Braços abaixados",
        "Ambos os braços levantados", "Braço direito levantado", "Braço esquerdo levantado", "Braços abaixados",
        "Braço esquerdo levantado", "Ambos os braços levantados", "Braço direito levantado", "Braços abaixados",
        "Braço direito levantado", "Ambos os braços levantados", "Braço esquerdo levantado", "Braços abaixados"
    ]

    labels = [
        "levantar_ambos", "levantar_esquerdo", "levantar_direito", "repouso",
        "levantar_ambos", "levantar_direito", "levantar_esquerdo", "repouso",
        "levantar_esquerdo", "levantar_ambos", "levantar_direito", "repouso",
        "levantar_direito", "levantar_ambos", "levantar_esquerdo", "repouso"
    ]

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

    # Criar um pipeline de transformação de texto e classificação
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())

    # Treinar o modelo
    model.fit(X_train, y_train)

    # Fazer previsões no conjunto de teste
    predicted_labels = model.predict(X_test)

    # Avaliar o modelo
    print(metrics.classification_report(y_test, predicted_labels, zero_division=0))

    # Classificar novos textos
    new_texts = movements_log
    predicted_new_labels = model.predict(new_texts)
    # Supondo que `predicted_new_labels` seja uma lista do mesmo tamanho que `movements_log`
    text_categorizador = [f"{movement}-{label}" for movement, label in zip(movements_log, predicted_new_labels)]

    # Transformando em uma única string, caso necessário
    text_completo = " ".join(text_categorizador)
    print(text_completo)
    return text_completo, predicted_new_labels

# Função para dividir o texto em segmentos menores para evitar o limite de 1024 tokens
def split_text(text, max_length):
    words = text.split()
    return [" ".join(words[i:i+max_length]) for i in range(0, len(words), max_length)]

# Função para resumir movimentos e emocoes
def resumidor(movements_log, emotions_log, labels):
    # Resumo das Ações Detectadas
    action_frequencies = {}
    for action in set(movements_log):
        action_frequencies[action] = movements_log.count(action) / len(movements_log) * 100
    action_summary = "Resumo das Ações Detectadas:\n"
    for action, frequency in action_frequencies.items():
        action_summary += f"- {action}: {frequency:.2f}% do tempo\n"

    # Resumo das Emoções Detectadas
    emotion_frequencies = {}
    for emotion in set(emotions_log):
        emotion_frequencies[emotion] = emotions_log.count(emotion) / len(emotions_log) * 100
    emotion_summary = "Resumo das Emoções Detectadas:\n"
    for emotion, frequency in emotion_frequencies.items():
        emotion_summary += f"- {emotion}: {frequency:.2f}% do tempo\n"

    # Resumo das Labels Detectadas
    labels_frequencies = {}
    for label in set(labels):
        labels_frequencies[label] = labels.count(label) / len(labels) * 100
    label_summary = "Resumo das Labels Detectadas:\n"
    for label, frequency in labels_frequencies.items():
        label_summary += f"- {label}: {frequency:.2f}% do tempo\n"

    # Resumo das Labels Detectadas
    labels_frequencies = {}  # Inicialize corretamente como um dicionário
    for label in set(labels):
        labels_frequencies[label] = labels.count(label) / len(labels) * 100

    label_summary = "Resumo das Labels Detectadas:\n"
    for label, frequency in labels_frequencies.items():
        label_summary += f"- {label}: {frequency:.2f}% do tempo\n"

    return action_summary, emotion_summary, label_summary

# Configurações principais
#script_dir = "/content/drive/MyDrive/Pós Tech - IA PARA DEVS/Pós Tech/FASE 4/"
script_dir = "/content/drive/MyDrive/FASE 4/"
image_folder = os.path.join(script_dir, 'images')
known_face_encodings, known_face_names = load_images_from_folder(image_folder)

input_video_path = os.path.join(script_dir, 'video-tech.mp4')
output_video_path = os.path.join(script_dir, 'output_video_recognize.mp4')

movements_log, emotions_log = detect_faces_and_emotions(input_video_path, output_video_path, known_face_encodings, known_face_names)
text_completo, labels = categorizador(movements_log)

#print(resumidor(text_categorizador, 200, 50, do_sample=True))
action_summary, emotion_summary, label_summary = resumidor(movements_log, emotions_log, list(labels))
print(action_summary)
print(emotion_summary)
print(label_summary)